In [19]:
#import libraries
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import gradio as gr
import os
import plotly.graph_objects as go

In [20]:
#define predictPPM 
def predictPPM(df, split):
    ts= pd.read_csv('datappm.csv')
    df2 =ts.copy()
    ttSplit=split/100
    ts['Date']=pd.to_datetime(ts['Date'])
    ts.rename(columns={'#PPM':'PPM'},inplace=True)
    ts=ts.set_index(['Date'])
    ts['months'] = [x.month for x in ts.index]
    ts['years'] = [x.year for x in ts.index]
    ts.reset_index(drop=True, inplace=True)

    # Split Data
    X=ts.drop("PPM",axis=1)
    Y= ts["PPM"]
    X_train=X[:int (len(Y)*ttSplit)] 
    X_test=X[int(len(Y)*ttSplit):]
    Y_train=Y[:int (len(Y)*ttSplit)] 
    Y_test=Y[int(len(Y)*ttSplit):]

    # fit the model
    rf = RandomForestRegressor()
    rf.fit(X_train, Y_train)
    
    df1=df2.set_index(['Date'])
    df1.rename(columns={'#PPM':'PPM'},inplace=True)
    train=df1.PPM[:int (len(ts.PPM)*ttSplit)]
    test=df1.PPM[int(len(ts.PPM)*ttSplit):]
    preds=rf.predict(X_test).astype(int) 
    predictions=pd.DataFrame(preds,columns=['PPM'])
    predictions.index=test.index
    predictions.reset_index(inplace=True)
    predictions['Date']=pd.to_datetime(predictions['Date'])
    print(predictions)
    
    #combine all into one table
    ts_df=df
    train= ts_df[:int (len(ts_df)*ttSplit)]
    test= ts_df[int(len(ts_df)*ttSplit):] 

    df2['Date']=pd.to_datetime(df2['Date'])
    df2.rename(columns={'#PPM':'PPM'},inplace=True)
    df3= predictions
    df2['origin']='status '
    df3['origin']='prediction'
    df4=pd.concat([df2, df3])
    print(df4)
    return df4
#define demo of gradio
demo = gr.Interface(
    fn =predictPPM,inputs = [gr.UploadButton(label="Input data for PPM TimeSeries"),
        gr.Slider(1, 100, value=75, step=1, label="Train test split percentage"),
    ],
    outputs=gr.LinePlot(x='Date', y='PPM', color='origin')
)
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\M\AppData\Local\Temp\ipykernel_13156\3567669655.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts['Date']=pd.to_datetime(ts['Date'])
C:\Users\M\AppData\Local\Temp\ipykernel_13156\3567669655.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  predictions['Date']=pd.to_datetime(predictions['Date'])
C:\Users\M\AppData\Local\Temp\ipykernel_13156\3567669655.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['Date']=pd.to_datetime(df2['Date'])


         Date   PPM
0  2018-10-01  2254
1  2018-11-01  2249
2  2018-12-01  2253
3  2019-01-01  3108
4  2019-02-01  3085
..        ...   ...
64 2024-02-01  3085
65 2024-03-01  2831
66 2024-04-01  2793
67 2024-05-01  2483
68 2024-06-01  1560

[69 rows x 2 columns]
         Date    PPM      origin
0  2001-09-01  31610     status 
1  2001-10-01  36168     status 
2  2001-11-01  37043     status 
3  2001-12-01  27614     status 
4  2002-01-01  21201     status 
..        ...    ...         ...
64 2024-02-01   3085  prediction
65 2024-03-01   2831  prediction
66 2024-04-01   2793  prediction
67 2024-05-01   2483  prediction
68 2024-06-01   1560  prediction

[342 rows x 3 columns]


C:\Users\M\AppData\Local\Temp\ipykernel_13156\3567669655.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ts['Date']=pd.to_datetime(ts['Date'])
C:\Users\M\AppData\Local\Temp\ipykernel_13156\3567669655.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  predictions['Date']=pd.to_datetime(predictions['Date'])
C:\Users\M\AppData\Local\Temp\ipykernel_13156\3567669655.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df2['Date']=pd.to_datetime(df2['Date'])


         Date   PPM
0  2016-07-01  6736
1  2016-08-01  6947
2  2016-09-01  7127
3  2016-10-01  7064
4  2016-11-01  7240
..        ...   ...
91 2024-02-01  6757
92 2024-03-01  6482
93 2024-04-01  7157
94 2024-05-01  7640
95 2024-06-01  6770

[96 rows x 2 columns]
         Date    PPM      origin
0  2001-09-01  31610     status 
1  2001-10-01  36168     status 
2  2001-11-01  37043     status 
3  2001-12-01  27614     status 
4  2002-01-01  21201     status 
..        ...    ...         ...
91 2024-02-01   6757  prediction
92 2024-03-01   6482  prediction
93 2024-04-01   7157  prediction
94 2024-05-01   7640  prediction
95 2024-06-01   6770  prediction

[369 rows x 3 columns]
Keyboard interruption in main thread... closing server.
